## Setup Environment

In [8]:
from cassandra.cluster import Cluster
from cassandra.query import dict_factory

import cred_cassandra as c

cluster = Cluster([c.host])
session = cluster.connect(keyspace='tourdelight')
session.row_factory = dict_factory

## Setup Tables and Data

In [9]:
def pretty_resultset_table(result_set):
    rows = [row for row in result_set]

    if not rows:
        return "No data available"

    column_names = rows[0].keys()

    column_widths = {col: max(len(str(col)), max(len(str(row[col])) for row in rows)) for col in column_names}

    header = " | ".join(f"{col.ljust(column_widths[col])}" for col in column_names)
    separator = "-+-".join("-" * column_widths[col] for col in column_names)

    formatted_rows = []
    for row in rows:
        formatted_row = " | ".join(str(row[col]).ljust(column_widths[col]) for col in column_names)
        formatted_rows.append(formatted_row)

    table = "\n".join([header, separator] + formatted_rows)
    print(table)


def drop_all_tables():
    query = "SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'tourdelight';"
    rows = session.execute(query)
    for row in rows:
        query = f"DROP TABLE IF EXISTS {row["table_name"]};"
        session.execute(query)


def init_tables():
    query = """
    CREATE TABLE IF NOT EXISTS region (
    rid int,
    name text,
    primary key (rid)
    );
    """
    session.execute(query)

    query = """
    CREATE TABLE IF NOT EXISTS tour (
    tid int,
    descr text,
    duration int,
    distance int,
    ascent text,
    descent text,
    primary key (tid)
    )
    """
    session.execute(query)

    query = """
    CREATE TABLE IF NOT EXISTS user (
    uid int,
    email text,
    name text,
    city text,
    country text,
    primary key (uid)
    )
    """
    session.execute(query)

    query = """
    CREATE TABLE IF NOT EXISTS tour_by_region (
    tid int,
    rid int,
    r_name text,
    t_descr text,
    t_duration int,
    t_distance int,
    primary key (rid, tid)
    )
    """
    session.execute(query)

    query = """
    CREATE TABLE IF NOT EXISTS tour_by_user_rating (
    tid int,
    uid int,
    rating int,
    primary key (tid, uid, rating)
    )
    WITH CLUSTERING ORDER BY (uid ASC, rating DESC)
    """
    session.execute(query)

    query = """
    CREATE TABLE IF NOT EXISTS tour_by_diff_activity (
    tid int,
    rid int,
    difficulty text,
    activity text,
    t_descr text,
    t_duration int,
    t_distance int,
    primary key ((tid, rid), activity, difficulty)
    )
    """
    session.execute(query)


def fill_example_values():
    # fill regions
    queries = [
        "INSERT INTO region(rid, name) VALUES (1, 'Schrammsteine')",
        "INSERT INTO region(rid, name) VALUES (2, 'Lilienstein')",
        "INSERT INTO region(rid, name) VALUES (3, 'Festung Königstein')",
        "INSERT INTO region(rid, name) VALUES (4, 'Bastei')",
        "INSERT INTO region(rid, name) VALUES (5, 'Pfaffenstein')"
    ]

    # fill tours
    queries.extend([
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(1, '50.9335, 14.15', '50.9340, 14.18', 'Leichte Wanderung durch die Schrammsteine', 2500, 90);",
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(2, '50.9342, 14.16', '50.9360, 14.20', 'Felsenpfad zum Aussichtspunkt in den Schrammsteinen', 3200, 120);",
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(3, '50.9195, 14.08', '50.9200, 14.11', 'Rundtour um den Lilienstein', 4000, 140);",
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(4, '50.9180, 14.10', '50.9205, 14.12', 'Aufstieg zur Spitze des Liliensteins', 1800, 60);",
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(5, '50.9233, 14.05', '50.9240, 14.07', 'Historische Tour zur Festung Königstein', 2700, 100);",
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(6, '50.9240, 14.04', '50.9260, 14.06', 'Panoramaweg rund um die Festung Königstein', 3400, 120);",
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(7, '50.9675, 14.08', '50.9680, 14.10', 'Wanderung zur Bastei-Brücke', 2000, 70);",
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(8, '50.9650, 14.07', '50.9670, 14.09', 'Ausgiebige Bastei-Rundtour', 4500, 160);",
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(9, '50.9285, 14.22', '50.9300, 14.25', 'Wanderung zum Pfaffenstein', 2300, 85);",
        "INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES(10, '50.9290, 14.23', '50.9310, 14.26', 'Höhenpfad mit Blick auf den Pfaffenstein', 3000, 110);"
    ])

    # fill user
    queries.extend([
        "INSERT INTO user(uid, city, country, email, name) VALUES(1, 'Dresden', 'Deutschland', 'maria.schmidt@example.com', 'Maria Schmidt');",
        "INSERT INTO user(uid, city, country, email, name) VALUES(2, 'Pirna', 'Deutschland', 'lars.mueller@example.com', 'Lars Müller');",
        "INSERT INTO user(uid, city, country, email, name) VALUES(3, 'Leipzig', 'Deutschland', 'anja.bauer@example.com', 'Anja Bauer');",
        "INSERT INTO user(uid, city, country, email, name) VALUES(4, 'Berlin', 'Deutschland', 'thomas.schulz@example.com', 'Thomas Schulz');",
        "INSERT INTO user(uid, city, country, email, name) VALUES(5, 'Hamburg', 'Deutschland', 'julia.fischer@example.com', 'Julia Fischer');",
        "INSERT INTO user(uid, city, country, email, name) VALUES(6, 'Köln', 'Deutschland', 'marc.wagner@example.com', 'Marc Wagner');",
        "INSERT INTO user(uid, city, country, email, name) VALUES(7, 'Stuttgart', 'Deutschland', 'sabrina.mayer@example.com', 'Sabrina Mayer');",
        "INSERT INTO user(uid, city, country, email, name) VALUES(8, 'München', 'Deutschland', 'alex.klein@example.com', 'Alex Klein');",
        "INSERT INTO user(uid, city, country, email, name) VALUES(9, 'Frankfurt', 'Deutschland', 'nina.hoffmann@example.com', 'Nina Hoffmann');",
        "INSERT INTO user(uid, city, country, email, name) VALUES(10, 'Bremen', 'Deutschland', 'robert.weber@example.com', 'Robert Weber');"
    ])

    # fill tour by region
    queries.extend([
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(1, 1, 'Schrammsteine', 'Leichte Wanderung durch die Schrammsteine', 2500, 90);",
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(1, 2, 'Schrammsteine', 'Felsenpfad zum Aussichtspunkt in den Schrammsteinen', 3200, 120);",
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(2, 3, 'Lilienstein', 'Rundtour um den Lilienstein', 4000, 140);",
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(2, 4, 'Lilienstein', 'Aufstieg zur Spitze des Liliensteins', 1800, 60);",
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(3, 5, 'Festung Königstein', 'Historische Tour zur Festung Königstein', 2700, 100);",
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(3, 6, 'Festung Königstein', 'Panoramaweg rund um die Festung Königstein', 3400, 120);",
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(4, 7, 'Bastei', 'Wanderung zur Bastei-Brücke', 2000, 70);",
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(4, 8, 'Bastei', 'Ausgiebige Bastei-Rundtour', 4500, 160);",
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(5, 9, 'Pfaffenstein', 'Wanderung zum Pfaffenstein', 2300, 85);",
        "INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES(5, 10, 'Pfaffenstein', 'Höhenpfad mit Blick auf den Pfaffenstein', 3000, 110);"
    ])

    # fill tour by user rating
    queries.extend([
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(1, 1, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(1, 2, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(1, 3, 3);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(2, 4, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(2, 5, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(2, 6, 3);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(3, 7, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(3, 8, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(3, 9, 3);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(4, 10, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(4, 1, 3);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(4, 2, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(5, 3, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(5, 4, 3);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(5, 5, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(6, 6, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(6, 7, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(6, 8, 3);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(7, 9, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(7, 10, 3);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(7, 1, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(8, 2, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(8, 3, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(8, 4, 3);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(9, 5, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(9, 6, 3);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(9, 7, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(10, 8, 4);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(10, 9, 5);",
        "INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES(10, 10, 3);"
    ])

    # fill tour by difficulty and activity 
    queries.extend([
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(1, 1, 'leicht', 'wanderung', 'Leichte Wanderung durch die Schrammsteine', 90, 2500);",
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(1, 2, 'mittel', 'wanderung', 'Felsenpfad zum Aussichtspunkt in den Schrammsteinen', 120, 3200);",
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(2, 3, 'leicht', 'wanderung', 'Rundtour um den Lilienstein', 140, 4000);",
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(2, 4, 'mittel', 'bergwanderung', 'Aufstieg zur Spitze des Liliensteins', 60, 1800);",
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(3, 5, 'mittel', 'bergwanderung', 'Historische Tour zur Festung Königstein', 100, 2700);",
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(3, 6, 'schwer', 'bergwanderung', 'Panoramaweg rund um die Festung Königstein', 120, 3400);",
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(4, 7, 'leicht', 'wanderung', 'Wanderung zur Bastei-Brücke', 70, 2000);",
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(4, 8, 'schwer', 'bergwanderung', 'Ausgiebige Bastei-Rundtour', 160, 4500);",
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(5, 9, 'mittel', 'wanderung', 'Wanderung zum Pfaffenstein', 85, 2300);",
        "INSERT INTO tour_by_diff_activity(rid, tid, difficulty, activity, t_descr, t_duration, t_distance) VALUES(5, 10, 'schwer', 'bergwanderung', 'Höhenpfad mit Blick auf den Pfaffenstein', 110, 3000);"
    ])

    for query in queries:
        #print(f"fill_example_value: running query: {query}")
        session.execute(query)



In [10]:
drop_all_tables()
init_tables()
fill_example_values()

## Add New Entry Functions

In [11]:
def add_new_region(rid, name):
    query = f"INSERT INTO region(rid, name) VALUES({rid}, '{name}');"
    session.execute(query)


def add_new_tour(tid, ascent, descent, descr, distance, duration, rid, r_name, activity, difficulty):
    query = f"INSERT INTO tour(tid, ascent, descent, descr, distance, duration) VALUES({tid}, '{ascent}', '{descent}', '{descr}', {distance}, {duration});"
    session.execute(query)

    query = f"INSERT INTO tour_by_region(rid, tid, r_name, t_descr, t_distance, t_duration) VALUES ({rid}, {tid}, '{r_name}', '{descr}', {distance}, {duration});"
    session.execute(query)

    query = f"INSERT INTO tour_by_diff_activity(tid, rid, activity, difficulty, t_descr, t_distance, t_duration) VALUES ({tid}, {rid}, '{activity}', '{difficulty}', '{descr}', {distance}, {duration});"
    session.execute(query)


def add_new_user_rating(tid, uid, rating):
    query = f"INSERT INTO tour_by_user_rating(tid, uid, rating) VALUES({tid}, {uid}, {rating});"
    session.execute(query)


def add_new_user(uid, city, country, email, name):
    query = f"INSERT INTO user(uid, city, country, email, name) VALUES({uid}, '{city}', '{country}', '{email}', '{name}');"
    session.execute(query)


def get_tables():
    query = "SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'tourdelight';"
    rows = session.execute(query)
    for row in rows:
        print(f"\n{row['table_name']}:")
        query = f"SELECT * FROM {row['table_name']}"";"
        pretty_resultset_table(session.execute(query))


def get_table(table_name):
    print(f"\n{table_name}:")
    query = f"SELECT * FROM {table_name};"
    pretty_resultset_table(session.execute(query))


## Update Entry Functions

In [12]:
def update_tour_description(tid, rid, descr, activity, difficulty):
    query = f"UPDATE tour SET descr = '{descr}' WHERE tid = {tid};"
    session.execute(query)

    query = f"UPDATE tour_by_region SET t_descr = '{descr}' WHERE tid = {tid} AND rid = {rid};"
    session.execute(query)

    query = f"UPDATE tour_by_diff_activity SET t_descr = '{descr}' WHERE tid = {tid} AND rid = {rid} AND activity = '{activity}' AND difficulty = '{difficulty}';"
    session.execute(query)


## Print All Tables

In [13]:
get_tables()


region:
rid | name              
----+-------------------
5   | Pfaffenstein      
1   | Schrammsteine     
2   | Lilienstein       
4   | Bastei            
3   | Festung Königstein

tour:
tid | ascent         | descent        | descr                                               | distance | duration
----+----------------+----------------+-----------------------------------------------------+----------+---------
5   | 50.9233, 14.05 | 50.9240, 14.07 | Historische Tour zur Festung Königstein             | 2700     | 100     
10  | 50.9290, 14.23 | 50.9310, 14.26 | Höhenpfad mit Blick auf den Pfaffenstein            | 3000     | 110     
1   | 50.9335, 14.15 | 50.9340, 14.18 | Leichte Wanderung durch die Schrammsteine           | 2500     | 90      
8   | 50.9650, 14.07 | 50.9670, 14.09 | Ausgiebige Bastei-Rundtour                          | 4500     | 160     
2   | 50.9342, 14.16 | 50.9360, 14.20 | Felsenpfad zum Aussichtspunkt in den Schrammsteinen | 3200     | 120     
4   | 50.91

## Add new Entries

In [14]:
add_new_region(6, "Tierpark")
add_new_tour(tid=11, ascent="52.4923, 13.5241", descent="52.4944, 13.5254", descr="HTW Berlin Straße zur Mensa",
             distance="300", duration="5", rid=6, r_name="Tierpark", activity="wanderung", difficulty="leicht")

add_new_user(11, "Berlin", "Deutschland", "<EMAIL>", "Marie Mustermann")
add_new_user_rating(11, 11, 5)

get_table("tour")
update_tour_description(tid=11, rid=6, descr="Andere Beschreibung", activity="wanderung", difficulty="leicht")


tour:
tid | ascent           | descent          | descr                                               | distance | duration
----+------------------+------------------+-----------------------------------------------------+----------+---------
5   | 50.9233, 14.05   | 50.9240, 14.07   | Historische Tour zur Festung Königstein             | 2700     | 100     
10  | 50.9290, 14.23   | 50.9310, 14.26   | Höhenpfad mit Blick auf den Pfaffenstein            | 3000     | 110     
11  | 52.4923, 13.5241 | 52.4944, 13.5254 | HTW Berlin Straße zur Mensa                         | 300      | 5       
1   | 50.9335, 14.15   | 50.9340, 14.18   | Leichte Wanderung durch die Schrammsteine           | 2500     | 90      
8   | 50.9650, 14.07   | 50.9670, 14.09   | Ausgiebige Bastei-Rundtour                          | 4500     | 160     
2   | 50.9342, 14.16   | 50.9360, 14.20   | Felsenpfad zum Aussichtspunkt in den Schrammsteinen | 3200     | 120     
4   | 50.9180, 14.10   | 50.9205, 14.12   | Aufst

In [15]:
get_table("tour")


tour:
tid | ascent           | descent          | descr                                               | distance | duration
----+------------------+------------------+-----------------------------------------------------+----------+---------
5   | 50.9233, 14.05   | 50.9240, 14.07   | Historische Tour zur Festung Königstein             | 2700     | 100     
10  | 50.9290, 14.23   | 50.9310, 14.26   | Höhenpfad mit Blick auf den Pfaffenstein            | 3000     | 110     
11  | 52.4923, 13.5241 | 52.4944, 13.5254 | Andere Beschreibung                                 | 300      | 5       
1   | 50.9335, 14.15   | 50.9340, 14.18   | Leichte Wanderung durch die Schrammsteine           | 2500     | 90      
8   | 50.9650, 14.07   | 50.9670, 14.09   | Ausgiebige Bastei-Rundtour                          | 4500     | 160     
2   | 50.9342, 14.16   | 50.9360, 14.20   | Felsenpfad zum Aussichtspunkt in den Schrammsteinen | 3200     | 120     
4   | 50.9180, 14.10   | 50.9205, 14.12   | Aufst